In [17]:
import pandas as pd
import numpy as np
from scipy import stats

# 交互数据

## 交互数据单方差分析

In [18]:
stat =  pd.read_csv("./stat_origin.csv")
stat.head()

,subject_id,task_round,mouse_distance,mouse_click,keyboard_press,duration
0,2,0,137703,350,395,761.563
1,2,1,16718,8,1092,341.584
2,3,0,193067,385,430,852.177
3,3,1,16922,20,785,397.007
4,4,0,269443,470,382,1143.378


In [19]:
def anova(df: pd.DataFrame ,column: str):
    return stats.f_oneway(df[column][df['task_round'] == 0], df[column][df['task_round'] == 1])

In [20]:
stat_anova_res = pd.DataFrame(columns=['样本数据', 'F', 'p'])
stat_anova_res

,样本数据,F,p


In [21]:
for column in stat.columns.drop(["subject_id", "task_round"]):
    F, p = anova(stat, column)
    # stat_anova_res = stat_anova_res.append({'样本数据': column, 'F': F, 'p': p})
    stat_anova_res = pd.concat([stat_anova_res, pd.DataFrame.from_records([{'样本数据': column, 'F': F, 'p': p}])])
stat_anova_res['样本数据'] = stat_anova_res['样本数据'].map({"mouse_distance": "鼠标移动距离", "mouse_click": "鼠标点击次数", "keyboard_press": "键盘按键次数", "duration": "任务时长"})
stat_anova_res["F"] =stat_anova_res["F"].apply('{:.3f}'.format)
stat_anova_res["p"] =stat_anova_res["p"].apply('{:.3e}'.format)
stat_anova_res.to_csv("./stat_anova.csv", index=False)

## 交互数据正态性检验

In [22]:
def shapiro_wilk(df: pd.DataFrame ,column: str):
    round_0_res =  stats.shapiro(df[column][df['task_round'] == 0])
    round_1_res =  stats.shapiro(df[column][df['task_round'] == 1])
    return pd.DataFrame.from_records([{'样本数据': column, 'task_round': 0, 'W': round_0_res[0], 'p': round_0_res[1]}, {'样本数据': column, 'task_round': 1, 'W': round_1_res[0], 'p': round_1_res[1]}])
res = pd.DataFrame(columns=['样本数据', 'task_round', 'W', 'p'])
for column in stat.columns.drop(["subject_id", "task_round"]):
    res = pd.concat([res, shapiro_wilk(stat, column)])


res["样本数据"] = res["样本数据"].map({"mouse_distance": "鼠标移动距离", "mouse_click": "鼠标点击次数", "keyboard_press": "键盘按键次数", "duration": "任务时长"})
res["task_round"] = res["task_round"].map({0: "WIMP", 1: "M-CAT"})
res.rename(columns={"W": "W 值", "p": "p 值", "task_round": "任务组", "样本数据": "样本数据"}, inplace=True)

res = res.assign(通过检验 = lambda x: x["p 值"] > 0.05 )
res["通过检验"] = res["通过检验"].map({True: "通过", False: "拒绝"})

res.to_csv("./stat_shapiro_wilk.csv", index=False, float_format='%.4f')

In [23]:
stats.shapiro(stat["mouse_distance"][stat['task_round'] == 0])

ShapiroResult(statistic=0.9246721863746643, pvalue=0.22686441242694855)

## 交互数据组内方差齐性检验

In [24]:
def levene(df: pd.DataFrame ,column: str):
    return stats.levene(df[column][df['task_round'] == 0], df[column][df['task_round'] == 1])

In [25]:
stat_levene_res = pd.DataFrame(columns=['样本数据', 'F', 'p'])
for column in stat.columns.drop(["subject_id", "task_round"]):
    F, p = levene(stat, column)
    stat_levene_res = pd.concat([stat_levene_res, pd.DataFrame.from_records([{'样本数据': column, 'F': F, 'p': p}])])
stat_levene_res = stat_levene_res.assign(通过检验 = lambda x: x["p"] > 0.05 )

In [26]:
stat_levene_res["样本数据"] = stat_levene_res["样本数据"].map({"mouse_distance": "鼠标移动距离", "mouse_click": "鼠标点击次数", "keyboard_press": "键盘按键次数", "duration": "任务时长"})
stat_levene_res["F"] =stat_levene_res["F"].apply('{:.3f}'.format)
stat_levene_res["p"] =stat_levene_res["p"].apply('{:.5f}'.format)
stat_levene_res["通过检验"] = stat_levene_res["通过检验"].map({True: "通过", False: "拒绝"})
stat_levene_res.to_csv("./stat_levene.csv", index=False)
stat_levene_res

,样本数据,F,p,通过检验
0,鼠标移动距离,14.272,0.00076,拒绝
0,鼠标点击次数,15.696,0.00047,拒绝
0,键盘按键次数,5.652,0.02450,拒绝
0,任务时长,1.316,0.26096,通过


## 交互数据组间差异检验

由于鼠标移动距离有一组样本未通过正态性检验，将联合采用 Welch’s t
检验和 Kruskal-Wallis 单因素方差分析综合分析。Welch’s t 检验是一种参数检验方法，用对比较均值差异，但 Welch’s t 也支持近似正态分布的检验，但结果可能不够准确。Kruskal-Wallis 单因素方差分析是一种非参数检验方法，用于检验样本间中位数差异，不要求总体分布参数。

鼠标点击次数和键盘按键次数通过了正态性检验，但未通过方差齐性检验，因此采用 Welch’s t 检验。Welch’s t 检验不要求样本间方差相同。

任务时间通过了正态性检验和方差齐性检验，使用 ANOVA 单因素方差分析进行检验。

### 鼠标移动距离（Wt 和 KW）

In [27]:
mouse_distance_0 = stat["mouse_distance"][stat['task_round'] == 0]
mouse_distance_1 = stat["mouse_distance"][stat['task_round'] == 1]

In [28]:
mouse_distance_t, mouse_distance_p_0 =stats.ttest_ind(mouse_distance_0, mouse_distance_1, equal_var=False)
mouse_distance_H, mouse_distance_p_1 = stats.kruskal(mouse_distance_0, mouse_distance_1)
print(f"鼠标移动距离的 t 检验结果为：t={mouse_distance_t:.3f}, p={mouse_distance_p_0:.3e}")
print(f"鼠标移动距离的 Kruskal 检验结果为：H={mouse_distance_H:.3f}, p={mouse_distance_p_1:.3e}")

鼠标移动距离的 t 检验结果为：t=11.254, p=1.051e-08
鼠标移动距离的 Kruskal 检验结果为：H=21.774, p=3.067e-06


### 鼠标点击次数 键盘击键次数（Wt）

In [29]:
mouse_click_t, mouse_click_p = stats.ttest_ind(stat["mouse_click"][stat['task_round'] == 0], stat["mouse_click"][stat['task_round'] == 1], equal_var=False)
print(f"鼠标击键次数的 t 检验结果为：t={mouse_click_t:.3f}, p={mouse_click_p:.3e}")

鼠标击键次数的 t 检验结果为：t=17.451, p=4.887e-11


In [30]:
keyboard_press_t, keyboard_press_p = stats.ttest_ind(stat["keyboard_press"][stat['task_round'] == 0], stat["keyboard_press"][stat['task_round'] == 1], equal_var=False)
print(f"键盘按键次数的 t 检验结果为：t={keyboard_press_t:.3f}, p={keyboard_press_p:.3e}")

键盘按键次数的 t 检验结果为：t=-9.502, p=1.407e-08


### 任务时间（ANOVA）

### 汇总结果

In [31]:
F, p = stats.f_oneway(stat["duration"][stat['task_round'] == 0], stat["duration"][stat['task_round'] == 1])
print(f"任务时长的 ANOVA 检验结果为：~$t = {F:.3f}, p = {p:.3e}$~")

任务时长的 ANOVA 检验结果为：~$t = 39.127, p = 9.237e-07$~


In [37]:
res = pd.DataFrame(columns=['样本数据','检验方法', '统计量','值', 'p'])
res.loc[0] = ["鼠标移动距离", "Welch's t","t", mouse_distance_t, mouse_distance_p_0]
res.loc[1] = ["鼠标移动距离", "Kruskal-Wallis","H", mouse_distance_H, mouse_distance_p_1]
res.loc[2] = ["鼠标点击次数", "Welch's t","t", mouse_click_t, mouse_click_p]
res.loc[3] = ["键盘按键次数", "Welch's t","t", keyboard_press_t, keyboard_press_p]
res.loc[4] = ["任务时长", "ANOVA","F", F, p]
res = res.assign(pas="通过")
res["p"] = res["p"].apply('{:.3e}'.format)
res["值"] = res["值"].apply('{:.3f}'.format)
res = res.rename({"pas": "通过（p<0.05）"}, axis=1)
res.to_csv("stat_result.csv", index=False, float_format='%.3f')
res


,样本数据,检验方法,统计量,值,p,通过（p<0.05）
0,鼠标移动距离,Welch's t,t,11.254,1.051e-08,通过
1,鼠标移动距离,Kruskal-Wallis,H,21.774,3.067e-06,通过
2,鼠标点击次数,Welch's t,t,17.451,4.887e-11,通过
3,键盘按键次数,Welch's t,t,-9.502,1.407e-08,通过
4,任务时长,ANOVA,F,11.380,2.187e-03,通过


# 量表数据

## 量表数据正态性检验

In [33]:
scale = pd.read_csv("./scale_origin.csv")
scale.head()

,subject_id,task_round,sus,effect,easy,easy_learn,satisfaction
0,13,1,87.5,47.6,67.2,24.6,48
1,13,0,47.5,36.4,37.2,13.0,28
2,15,1,80.0,54.6,64.8,26.6,48
3,15,0,55.0,42.8,47.0,19.0,34
4,16,1,72.5,45.8,58.4,26.6,49


In [34]:
res = pd.DataFrame(columns=['样本数据', 'task_round', 'W', 'p'])
for column in scale.columns.drop(["subject_id", "task_round"]):
    res = pd.concat([res, shapiro_wilk(scale, column)])
res = res.assign(通过检验 = lambda x: x["p"] > 0.05 )
res["task_round"] = res["task_round"].map({0: "WIMP", 1: "M-CAT"})
res["通过检验"] = res["通过检验"].map({True: "通过", False: "拒绝"})
res.rename(columns={"task_round": "任务组"}, inplace=True)

res["样本数据"] = res["样本数据"].map({"sus": "SUS 评分", "effect": "USE 有效性评分", "easy": "USE 易用性", "easy_learn": "USE 易学性评分", "satisfaction": "USE 满意度评分", "mouse_distance": "鼠标移动距离", "mouse_click": "鼠标点击次数", "keyboard_press": "键盘按键次数", "duration": "任务时长"})
res.to_csv("./scale_shapiro_wilk.csv", index=False, float_format='%.4f')
res

,样本数据,任务组,W,p,通过检验
0,SUS 评分,WIMP,0.967739,0.823279,通过
1,SUS 评分,M-CAT,0.943743,0.431777,通过
0,USE 有效性评分,WIMP,0.965602,0.788450,通过
1,USE 有效性评分,M-CAT,0.860291,0.024367,拒绝
0,USE 易用性,WIMP,0.950191,0.527465,通过
1,USE 易用性,M-CAT,0.884742,0.055871,通过
0,USE 易学性评分,WIMP,0.935842,0.332947,通过
1,USE 易学性评分,M-CAT,0.707486,0.000299,拒绝
0,USE 满意度评分,WIMP,0.905932,0.117297,通过
1,USE 满意度评分,M-CAT,0.813209,0.005459,拒绝


## 量表数据组间方差齐性检验

In [35]:
res = pd.DataFrame(columns=['样本数据', 'F', 'p'])
for column in scale.columns.drop(["subject_id", "task_round"]):
    F, p = levene(scale, column)
    res = pd.concat([res, pd.DataFrame({'样本数据': column, 'F': F, 'p': p}, index=[0])])
res = res.assign(通过检验 = lambda x: x["p"] > 0.05 )

res["通过检验"] = res["通过检验"].map({True: "通过", False: "拒绝"})
res["样本数据"] = res["样本数据"].map({"sus": "SUS 评分", "effect": "USE 有效性评分", "easy": "USE 易用性", "easy_learn": "USE 易学性评分", "satisfaction": "USE 满意度评分", "mouse_distance": "鼠标移动距离", "mouse_click": "鼠标点击次数", "keyboard_press": "键盘按键次数", "duration": "任务时长"})
res.to_csv("./scale_levene.csv", index=False, float_format='%.4f')

In [36]:
res

,样本数据,F,p,通过检验
0,SUS 评分,1.636804,0.211265,通过
0,USE 有效性评分,4.487491,0.043151,拒绝
0,USE 易用性,3.006638,0.093923,通过
0,USE 易学性评分,9.903864,0.003892,拒绝
0,USE 满意度评分,11.380226,0.002187,拒绝


## 量表数据推论统计

SUS 评分和 USE 易用性评分通过了正态性检验和方差齐性检验，使用 ANOVA 单因素方差分析检验均值差异。取显著性水平为 $\alpha = 0.05$，在 $p > 0.05$ 时通过检验。

USE 有效性、易学性和满意度均未通过正态性检验和方差齐性检验，联合采用 Welch’s t 检验和 Kruskal-Wallis 单因素方差分析综合分析组内差异。取显著性水平为 $\alpha = 0.05$，在 $p > 0.05$ 时通过检验。